In [34]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sentiment_analyzer import get_sentiment


In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
from textblob import TextBlob

# Load financial news
news_df = pd.read_csv('../data/raw_analyst_ratings.csv')

# Load stock prices
stock_df = pd.read_csv('../data/yfinance_data/AMZN_historical_data.csv')

In [30]:
print("News DataFrame:")
news_df.head()


News DataFrame:


,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [31]:
print("\nStock DataFrame:")
stock_df.head()


Stock DataFrame:


,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,1997-05-15,0.121875,0.125000,0.096354,0.097917,0.097917,1443120000,0.0,0.0
1,1997-05-16,0.098438,0.098958,0.085417,0.086458,0.086458,294000000,0.0,0.0
2,1997-05-19,0.088021,0.088542,0.081250,0.085417,0.085417,122136000,0.0,0.0
3,1997-05-20,0.086458,0.087500,0.081771,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21,0.081771,0.082292,0.068750,0.071354,0.071354,377064000,0.0,0.0


In [32]:
# For news
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')


# For stock prices
stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.date


In [35]:
news_df['sentiment'] = news_df['headline'].apply(get_sentiment)


In [ ]:
merged_df = pd.merge(
    news_df, 
    stock_df, 
    left_on=['stock', 'date'], 
    right_on=['Ticker', 'Date']
)
print(merged_df.head())

In [ ]:
correlation = merged_df[['sentiment', 'Close']].corr()
print("Correlation between sentiment and Close price:")
print(correlation)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.scatterplot(x='sentiment', y='Close', data=merged_df)
plt.title('Sentiment vs. Stock Close Price')
plt.xlabel('Sentiment Score')
plt.ylabel('Close Price')
plt.show()